In [ ]:
# default_exp generator

# Sequence request generator

> Генерируем формы для заказов на сиквенс в евроген или в геном.

In [ ]:
#hide
from nbdev.showdoc import *

Нужные импорты и переменные для подключения к базе и автоматического заполнения форм

In [ ]:
#export
import os
from airtable import airtable
import numpy as np
import pandas as pd
import xlsxwriter
import collections

base_key = os.environ['AIR_BASE_KEY']
table_name = os.environ['AIR_TABLE_NAME']
api_key = os.environ['AIR_API_KEY']

promoters = {'CMV':'c1for2', 
             'hPGK':'PGK F', 
             'PGK':'PGK F', 
             'EF1':'EF1 F', 
             'CAG':None, 
             'pCAG':None, 
             'SV40':'SV40 F'}

terminators = {'SV40':'c1rev2', 
               'BGH':'BGH R', 
               'rbGlob':'G R', 
               'PGK':'PGK R'}

conc = "5 mkM"

plasmida = "Плазмида"

Подключаемся к базе, используя имя базы и ключ. Страница (table_name) из базы читается в виде многоуровневого OrderedDict.

In [ ]:
#export
airtable = airtable.Airtable(base_key, api_key=api_key)

In [ ]:
assert type(airtable.airtable_url) == str, 'ConnectionError: не произошел коннект, возможно ключи к базам кривые'
assert type(airtable.get(table_name)['records'][12]) == collections.OrderedDict, 'Ochibka: почему-то таблица криво прочиталась'

Слеудющими действиями мы создаем Pandas.DataFrame таблицу, содержащую имена реакций плонирований и seqID номера. Это все, что нам нужно для формирования содержания формы для заказа секвенирования.

In [ ]:
#export
names = []
seqIDs = []

for r in airtable.iterate(table_name):
    if r['fields']:
        names.append(r['fields']['Name'])
        seqIDs.append('{:.2f}'.format(r['fields']['SeqID']))
        
pandaframe = pd.DataFrame({'Name':names, 'SeqID': seqIDs}).round(decimals=2)

In [ ]:
assert len(names) != 0, 'EmptyNames: в именах пусто почему-то'
assert len(seqIDs) != 0, 'EmptySeqID: в seqIDs пусто почему-то'